<center><a href="https://www.nvidia.com/dli"> <img src="https://developer-blogs.nvidia.com/wp-content/uploads/2020/03/DLI_Feature_new.png" alt="Header" style="width: 400px;"/> </a></center>

# 4b. Deploying Your Model
Now that we have a well trained model, it's time to use it. In this exercise, we'll expose new images to our model and detect the correct letters of the sign language alphabet. Let's get started!

## 4b.1 Objectives

* Load an already-trained model from disk
* Reformat images for a model trained on images of a different format
* Perform inference with new images, never seen by the trained model and evaluate its performance

In [ ]:
!pip install -q triton

In [ ]:
# Download utils.py
!gdown 1SseHfqVBRrRNyETVDJkXiDsmsS2HzwkD

In [ ]:
# Download trained model
!gdown 1rRozKpcChEtN4fHGMfYX-QNYCqRQHXUB

In [ ]:
!gdown 1UAbN8FBfJsQLeEx8uJcEEeS7SJf_mlab
!gdown 1t7-vjrVdgMIxf_ATC5i7kfWHPbPhnaa8

In [ ]:
!unzip asl_image.zip
!unzip asl_data.zip

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torchvision.io as tv_io
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

## 4b.2 Loading the Model
Now that we're in a new notebook, let's load the saved model that we trained. Our save from the previous exercise created a folder called "asl_model". We can load the model by selecting the same folder.

Since our model uses a [custom module](https://pytorch.org/tutorials/beginner/examples_nn/two_layer_net_module.html), we will need to load the code for that class. We have saved a copy of the code in [utils.py](./uitls.py).

In [ ]:
from utils import MyConvBlock

Now that we have a definition for `MyConvBlock`, we can use [torch.load](https://pytorch.org/docs/stable/generated/torch.load.html) to load a model from a path. We can use `map_location to specify the device. When we print the model, does it look the same as in the last notebook?

In [ ]:
model = torch.load('model.pth', map_location=device)
model

We can also verify if the model is on our GPU.

In [ ]:
next(model.parameters()).device

## 4b.3 Preparing an Image for the Model

It's now time to use the model to make predictions on new images that it's never seen before. This is also called inference. We have a set of images in the `data/asl_images` folder. Try opening it using the left navigation and explore the images.

You'll notice that the images we have are much higher resolution than the images in our dataset. They are also in color. Remember that our images in the dataset were 28x28 pixels and grayscale. It's important to keep in mind that whenever we make predictions with a model, the input must match the shape of the data that the model was trained on. For this model, the training dataset was of the shape: (27455, 28, 28, 1). This corresponded to 27455 images of 28 by 28 pixels each with one color channel (grayscale).

### 4b.3.1 Showing the Images

When we use our model to make predictions on new images, it will be useful to show the image as well. We can use the matplotlib library to do this.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image, cmap='gray')

In [ ]:
show_image('b.png')

### 4b.3.2 Scaling the Images

The images in our dataset were 28x28 pixels and grayscale. We need to make sure to pass the same size and grayscale images into our method for prediction. There are a few ways to edit images with Python, but TorchVision also has the [read_image](https://pytorch.org/vision/stable/generated/torchvision.io.read_image.html) function. We can let it know what kind of image to read with [ImageReadMode](https://pytorch.org/vision/stable/generated/torchvision.io.ImageReadMode.html#torchvision.io.ImageReadMode).

In [ ]:
image = tv_io.read_image('b.png', tv_io.ImageReadMode.GRAY)
image

Let's look at the shape of the image.

In [ ]:
image.shape

This image is much larger than what we trained on. We can use [TorchVision](https://pytorch.org/vision/stable/index.html)'s [Transforms](https://pytorch.org/vision/0.9/transforms.html) again to get the data in the form our model expects.

We will:
* Convert the image to float with [ToDtype](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.ToDtype.html)
  * We will set `scale` to `True` in order to convert from [0, 255] to [0, 1]
* [Resize](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.Resize.html#torchvision.transforms.v2.Resize) the image to be 28 x 28 pixels
* Convert the images to [Grayscale](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.Grayscale.html#torchvision.transforms.v2.Grayscale)
  * This step doesn't do anything since our models are already grayscale, but we've added it here to show an alternative way to get grayscale images.

In [ ]:
IMG_WIDTH = 28
IMG_HEIGHT = 28

preprocess_trans = transforms.Compose([
    transforms.ToDtype(torch.float32, scale=True), # Converts [0, 255] to [0, 1]
    transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
    transforms.Grayscale()  # From Color to Gray
])

Let's test `preprocess_trans` on an image to make sure it works correctly:

In [ ]:
processed_image = preprocess_trans(image)
processed_image

The numbers look correct, but how about the shape?

In [ ]:
processed_image.shape

Next, let's plot the image to see if it looks like what we trained on.

In [ ]:
plot_image = F.to_pil_image(processed_image)
plt.imshow(plot_image, cmap='gray')

Looking good! Let's pass it to our model.

### 4b.4 Making Predictions

Okay, now we're ready to predict! Our model still expects a batch of images. If the [squeeze](https://pytorch.org/docs/stable/generated/torch.squeeze.htmlhttps://pytorch.org/docs/stable/generated/torch.squeeze.html) removes dimensions of 1, [unsqueeze](https://pytorch.org/docs/stable/generated/torch.unsqueeze.htmlhttps://pytorch.org/docs/stable/generated/torch.unsqueeze.html) adds a dimension of 1 at the index we specify. The first dimension is usually the batch dimension, so we can say `.unsqueeze(0)`.

In [ ]:
batched_image = processed_image.unsqueeze(0)
batched_image.shape

Next, we should make sure the input tensor is on the same `device` as the model.

In [ ]:
batched_image_gpu = batched_image.to(device)
batched_image_gpu.device

Now we're ready to pass it to the model!

In [ ]:
output = model(batched_image_gpu)
output

### 4b.4.1 Understanding the Prediction

The predictions are in the format of a 24 length array. The larger the value, the more likely the input image belongs to the corresponding class. Let's make it a little more readable. We can start by finding which element of the array represents the highest probability. This can be done easily with the numpy library and the [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) function.

In [ ]:
prediction = output.argmax(dim=1).item()
prediction

Each element of the prediction array represents a possible letter in the sign language alphabet. Remember that j and z are not options because they involve moving the hand, and we're only dealing with still photos. Let's create a mapping between the index of the predictions array, and the corresponding letter.

In [ ]:
# Alphabet does not contain j or z because they require movement
alphabet = "abcdefghiklmnopqrstuvwxy"

We can now pass in our prediction index to find the corresponding letter.

In [ ]:
alphabet[prediction]

Let's put everything in a function so that we can make predictions just from the image file. Implement it in the function below using the functions and steps above.

In [ ]:
def predict_letter(file_path):
    show_image(file_path)
    image = tv_io.read_image(file_path, tv_io.ImageReadMode.GRAY)
    image = preprocess_trans(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    output = model(image)
    prediction = output.argmax(dim=1).item()
    # convert prediction to letter
    predicted_letter = alphabet[prediction]
    return predicted_letter

In [ ]:
predict_letter("b.png")

Let's also use the function with the 'a' letter in the asl_images datset:

In [ ]:
predict_letter("a.png")

## 4b.5 Summary

Great work on these exercises! You've gone through the full process of training a highly accurate model from scratch, and then using the model to make new and valuable predictions. If you have some time, we encourage you to take pictures with your webcam, upload them by dropping them into the data/asl_images folder, and test out the model on them. For Mac you can use Photo Booth. For windows you can select the Camera app from your start screen. We hope you try it. It's a good opportunity to learn some sign language! For instance, try out the letters of your name.

We can imagine how this model could be used in an application to teach someone sign language, or even help someone who cannot speak interact with a computer.

### 4b.5.1 Clear the Memory

Before moving on, please execute the following cell to clear up the GPU memory.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 4b.5.2 Next

We hope you've enjoyed these exercises. In the next sections we will learn how to take advantage of deep learning when we don't have a robust dataset available. See you there!
To learn more about inference on the edge, check out [this paper](http://web.eecs.umich.edu/~mosharaf/Readings/FB-ML-Edge.pdf) on the topic.

Now that we're familiar building your own models and have some understanding of how they work, we will turn our attention to the very powerful technique of using pre-trained models to expedite your work.

<center><a href="https://www.nvidia.com/dli"> <img src="https://developer-blogs.nvidia.com/wp-content/uploads/2020/03/DLI_Feature_new.png" alt="Header" style="width: 400px;"/> </a></center>